Импортируем нужные библиотеки

In [21]:
import pyomo.environ as pyo
import highspy
import openpyxl

Загрузка данных из Excel-файла

In [22]:
wb = openpyxl.load_workbook('feed_data.xlsx')
ws = wb.active

Создаём экземпляр конкретной модели Pyomo

In [23]:
model = pyo.ConcreteModel()

Создаём множества недель от 1 до 8, а также создаём множества ингредиентов

In [24]:
model.weeks = pyo.RangeSet(1, 8)
model.ingredients = pyo.Set(initialize=['Известняк', 'Зерно', 'Соевая_мука'])

Создаём переменных для решения x, которые представляют количество каждого ингредиента, используемого в каждую неделю(переменные ограничены неотрицательными значениями)

In [25]:
model.x = pyo.Var(model.weeks, model.ingredients, within=pyo.NonNegativeReals)

- Определение еженедельного расхода корма на одного цыпленка
- Определение содержания различных питательных веществ в ингредиентах
- Определение стоимости ингредиентов за фунт
- Определение требований(ограничений) к кормовому рациону.(считываем минимум из Excel таблицы)

In [26]:
model.feed_per_bird = {1: 0.26, 2: 0.48, 3: 0.75, 4: 1.00, 5: 1.30, 6: 1.60, 7: 1.90, 8: 2.10}

model.calcium = {'Известняк': 0.380, 'Зерно': 0.001, 'Соевая_мука': 0.002}
model.protein = {'Известняк': 0.00, 'Зерно': 0.09, 'Соевая_мука': 0.50}
model.fiber = {'Известняк': 0.00, 'Зерно': 0.02, 'Соевая_мука': 0.08}

model.cost_per_pound = {'Известняк': 0.12, 'Зерно': 0.45, 'Соевая_мука': 1.60}

model.min_calcium = int(ws.cell(row=2, column=2).value * 100)
model.max_calcium = 12
model.min_protein = int(float(ws.cell(row=3, column=2).value) * 100)
model.max_fiber = 5

Определение функции для вычисления общей стоимости кормового рациона, а также определение целевой функции, которая минимизирует общую стоимость

In [27]:
def total_cost(model):
    return sum(model.x[week, ingredient] * model.cost_per_pound[ingredient] for week in model.weeks for ingredient in model.ingredients)

model.objective = pyo.Objective(rule=total_cost, sense=pyo.minimize)

- Определение функции, которая обеспечивает, чтобы общее количество корма, использованное в каждую неделю, соответствовало еженедельному расходу корма на одного цыпленка
- Добавление ограничения на еженедельный расход корма
- Определение функций, которые обеспечивают соблюдение требований к содержанию питательных веществ в кормовом рационе
- Добавление ограничений на содержание питательных веществ

In [28]:
def total_feed_per_bird(model, week):
    return sum(model.x[week, ingredient] for ingredient in model.ingredients) == model.feed_per_bird[week] * 20000
model.feed_per_bird_limit = pyo.Constraint(model.weeks, rule=total_feed_per_bird)

def calcium_min(model, week):
    calcium_amount = sum(model.x[week, ingredient] * model.calcium[ingredient] for ingredient in model.ingredients)
    total_feed = sum(model.x[week, ingredient] for ingredient in model.ingredients)
    return calcium_amount >= model.min_calcium * total_feed / 100
model.calcium_limit = pyo.Constraint(model.weeks, rule=calcium_min)

def calcium_max(model, week):
    calcium_amount = sum(model.x[week, ingredient] * model.calcium[ingredient] for ingredient in model.ingredients)
    total_feed = sum(model.x[week, ingredient] for ingredient in model.ingredients)
    return calcium_amount <= model.max_calcium * total_feed / 100
model.calcium_max_limit = pyo.Constraint(model.weeks, rule=calcium_max)

def protein_min(model, week):
    protein_amount = sum(model.x[week, ingredient] * model.protein[ingredient] for ingredient in model.ingredients)
    total_feed = sum(model.x[week, ingredient] for ingredient in model.ingredients)
    return protein_amount >= model.min_protein * total_feed / 100
model.protein_limit = pyo.Constraint(model.weeks, rule=protein_min)

def fiber_max(model, week):
    fiber_amount = sum(model.x[week, ingredient] * model.fiber[ingredient] for ingredient in model.ingredients)
    total_feed = sum(model.x[week, ingredient] for ingredient in model.ingredients)
    return fiber_amount <= model.max_fiber * total_feed / 100
model.fiber_limit = pyo.Constraint(model.weeks, rule=fiber_max)

- Создание экземпляра solver, который будет использоваться для решения задачи оптимизации
- Решение оптимизационной задачи
- Получение значения целевой функции (общей стоимости)
- Итерация по неделям и вывод количества каждого ингредиента, используемого в каждую неделю

In [29]:
solver = pyo.SolverFactory('appsi_highs')
solver.solve(model)

total_cost_value = model.objective()
print(f"Общая стоимость кормового рациона: {total_cost_value:.2f}")

for week in model.weeks:
    print(f"Неделя {week}:")
    for ingredient in model.ingredients:
        print(f"{ingredient}: {model.x[week, ingredient].value:.2f} фунтов")
    print()

Общая стоимость кормового рациона: 148429.64
Неделя 1:
Известняк: 1627.42 фунтов
Зерно: 1566.56 фунтов
Соевая_мука: 2006.02 фунтов

Неделя 2:
Известняк: 3004.48 фунтов
Зерно: 2892.10 фунтов
Соевая_мука: 3703.42 фунтов

Неделя 3:
Известняк: 4694.49 фунтов
Зерно: 4518.91 фунтов
Соевая_мука: 5786.60 фунтов

Неделя 4:
Известняк: 6259.33 фунтов
Зерно: 6025.21 фунтов
Соевая_мука: 7715.46 фунтов

Неделя 5:
Известняк: 8137.12 фунтов
Зерно: 7832.78 фунтов
Соевая_мука: 10030.10 фунтов

Неделя 6:
Известняк: 10014.92 фунтов
Зерно: 9640.34 фунтов
Соевая_мука: 12344.74 фунтов

Неделя 7:
Известняк: 11892.72 фунтов
Зерно: 11447.90 фунтов
Соевая_мука: 14659.38 фунтов

Неделя 8:
Известняк: 13144.58 фунтов
Зерно: 12652.95 фунтов
Соевая_мука: 16202.47 фунтов

